In [1]:
import tropycal.tracks as tracks
import datetime as dt
import numpy as np
import pandas as pd

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/plot/plot.py:16: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tracks/plot.py:24: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/dataset.py:21: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn(warn_message)
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/plot.py:21: UserWarning: Warning: Cartopy is not installed in your python environm

In [2]:
import sys
sys.path.append('../')
from utils import data_processing as proc

In [3]:
from utils import data_processing as proc
#testing that the code works 
e, d= proc.prepare_tabular_data_vision(path="../data/last3years.csv", min_wind=50, min_steps=15,
                  max_steps=120, get_displacement=True, forecast=True, one_hot = True)

joining forecast data
--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (12.45 seconds)
timesteps of forecast for  joyce 29
no forecast for not_named 2018
timesteps of forecast for  kirk 33
no forecast for not_named 2018
timesteps of forecast for  leslie 84
timesteps of forecast for  rosa 34
no forecast for walaka 2018
timesteps of forecast for  sergio 61
timesteps of forecast for  michael 28
timesteps of forecast for  nadine 20
timesteps of forecast for  tara 16
timesteps of forecast for  vicente 18
timesteps of forecast for  willa 29
The dataframe of storms with forecast has been created.
one-hot added for  BASIN
df0 columns : Index(['SID', 'ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
       'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
       'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category',
       'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp',
       'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vma

In [6]:
#here is an example of how to use Tropycal 
#read north atlantic datasets
hurdat= tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)
#example to get forecast for one storm 
storm = hurdat.get_storm(('michael', 2018))
forecast = storm.get_operational_forecasts()
print('all available models', storm.get_operational_forecasts().keys()) 

# I chose models to use:
# #'HWRF','SHIP','NVGM','AEMI','NAM 
model_list = set(['HWRF','SHIP', 'AEMI','CLAP5','EMXI','CMC']).intersection(forecast.keys())

#This website has the explanation of all the models: 
#https://www.nhc.noaa.gov/modelsummary.shtml#:~:text=The%20National%20Hurricane%20Center%20 




--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (11.93 seconds)
all available models dict_keys(['CARQ', 'NAM', 'AC00', 'AEMN', 'AP01', 'AP02', 'AP03', 'AP04', 'AP05', 'AP06', 'AP07', 'AP08', 'AP09', 'AP10', 'AP11', 'AP12', 'AP13', 'AP14', 'AP15', 'AP16', 'AP17', 'AP18', 'AP19', 'AP20', 'AVNO', 'AVNX', 'CLP5', 'CTCX', 'DSHP', 'GFSO', 'HCCA', 'IVCN', 'IVDR', 'LGEM', 'OCD5', 'PRFV', 'SHF5', 'SHIP', 'TABD', 'TABM', 'TABS', 'TCLP', 'XTRP', 'CMC', 'NGX', 'UKX', 'AEMI', 'AHNI', 'AVNI', 'CEMN', 'CHCI', 'CTCI', 'DSPE', 'EGRR', 'LGME', 'NAMI', 'NEMN', 'RVCN', 'RVCX', 'SHPE', 'TBDE', 'TBME', 'TBSE', 'TVCA', 'TVCE', 'TVCN', 'TVCX', 'TVDG', 'UE00', 'UE01', 'UE02', 'UE03', 'UE04', 'UE05', 'UE06', 'UE07', 'UE08', 'UE09', 'UE10', 'UE11', 'UE12', 'UE13', 'UE14', 'UE15', 'UE16', 'UE17', 'UE18', 'UE19', 'UE20', 'UE21', 'UE22', 'UE23', 'UE24', 'UE25', 'UE26', 'UE27', 'UE28', 'UE29', 'UE30', 'UE31', 'UE32', 'UE33', 'UE34', 'UE35', 'UEMN', 'CEMI', 'CMCI', 'COTC', 'EGRI', 'HMON', '

In [83]:
#function to get forecast of hurricane based on name and year 
def get_forecast(hurdat, name, year, pred=24): #pred: hours prediction 
    try:
        storm = hurdat.get_storm((name, year))
        forecast = storm.get_operational_forecasts()
        #choose models 
        #"Ideally we are interested in SHIFOR5/SHIPS/GFS/EMXI/HWRF" - Leonard 
        model_list = set(['HWRF','SHIP','SHF5','GFS','EMXI','CLAP5','NAM','AP01','CMC']).intersection(forecast.keys())
        #create empty df 
        df_out = pd.DataFrame(columns=['datetime'])
        for model in model_list:  
            df_model = pd.DataFrame()
            for time in forecast[model]:    
                df = pd.DataFrame(forecast[model][time])
                temp = df.loc[df['fhr']==pred]
                #select columns
                temp = temp[['lat','lon','vmax','mslp']]
                temp = temp.add_prefix(str(model)+'_'+str(pred)+'_')
                temp['datetime'] = pd.to_datetime(time, format = '%Y%m%d%H')
                df_model = pd.concat([df_model, temp], axis=0)
            if df_model.shape[0]>0: 
                df_out = df_out.merge(df_model, on='datetime', how='outer')
        df_out = df_out.sort_values(by='datetime')         
    except:
        df_out = pd.DataFrame(columns=['datetime'])
    return df_out

def join_forecast(df, pred=24):
    print('joining forecast data')
    #create a dataframe to store joined data
    df_all= pd.DataFrame()

    # read hurdat data set for both basins: north atlantic and east pacific
    hurdat = tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)

    #get list of storm names and year
    df['NAME'] = df['NAME'].str.lower()
    df['YEAR'] = df['ISO_TIME'].dt.year
    storm_list = df[['SID','NAME','YEAR','BASIN']].drop_duplicates(['SID']) #get list of storm names and year
    storm_list.reset_index(inplace=True, drop=True)

    for i in range(len(storm_list)):
        SID = storm_list['SID'][i]
        name = storm_list['NAME'][i]
        year= storm_list['YEAR'][i]
        basin = storm_list['BASIN'][i]
        
        #get stat data for particular storm
        df_stat = df.loc[df['NAME']==name]
        
        #no forecast for these basins 
        if basin in ['SP','SI','NI','WP']:
            df_all = pd.concat([df_all, df_stat], axis=0)        
        else:  
            #try to get forecast for particular storm
            df_forecast = get_forecast(hurdat, name, year, pred)
            if df_forecast.shape[0]>0:
                print('timesteps of forecast for ',name, df_forecast.shape[0])
                #join with dataframe
                df_forecast['NAME']= name
                df_joined = df_stat.merge(df_forecast, how='left', left_on=['ISO_TIME','NAME'], right_on=['datetime','NAME'])
                df_joined.drop('datetime', axis=1, inplace=True)
                df_all = pd.concat([df_all, df_joined], axis=0)
            else:
                print('no forecast for', name, year)
                df_all = pd.concat([df_all, df_stat], axis=0)  
    #drop duplicated values 
    df1.drop_duplicates(subset=['SID','ISO_TIME'], keep='last')
    #sort values 
    df2.sort_values(by=['SID','ISO_TIME'], ascending=True)
    df_all.reset_index(inplace=True, drop=True)
    
    #drop added columns
    df_all.drop(['NAME','YEAR'], inplace=True, axis=1)
    
    print("The dataframe of storms with forecast has been created.")
    return df_all

In [94]:
#exposed version of the prepare_tabular_vision data 
#testing it works with the prepare_tabular_vision code 
min_wind=50
min_steps=15
max_steps=120 
get_displacement=True 
one_hot = False
#documentation on data https://www.ncdc.noaa.gov/ibtracs/pdf/IBTrACS_v04_column_documentation.pdf 
# path="../data/since1980.csv"
path="../data/last3years.csv"
data = pd.read_csv(path)
data.drop(0, axis=0, inplace=True) #drop secondary column names
# select interesting columns
df0 = data[['SID','NAME', 'BASIN','ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'STORM_DIR']]
# transform data from String to numeric
df0 = proc.numeric_data(df0)
# smooth cos & sign of day
df0 = proc.smooth_features(df0)
# # add wind category
df0['wind_category'] = df0.apply(lambda x: proc.sust_wind_to_cat_val(x['WMO_WIND']), axis=1)
# df0 = df0.iloc[15000:17000]

In [95]:
#crop out small section for testing 
df1 = df0.iloc[15000:16000]
df1 = join_forecast(df1, pred=24)
df2 = proc.add_one_hot(df1, df1['BASIN'],'_basin')
# df2.head()

joining forecast data
--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (15.55 seconds)
              SID       NAME  YEAR BASIN
0   2018255N37320      joyce  2018   NaN
1   2018258S10072  not_named  2018    SI
2   2018263N26249  not_named  2018    EP
3   2018264N14145      trami  2018    WP
4   2018265N08338       kirk  2018   NaN
5   2018265N13308  not_named  2018   NaN
6   2018265N35315     leslie  2018   NaN
7   2018268N14253       rosa  2018    EP
8   2018269N29151  not_named  2018    WP
9   2018270N12218     walaka  2018    EP
10  2018270S11162       liua  2018    SP
11  2018271N07151   kong-rey  2018    WP
12  2018273N12259     sergio  2018    EP
13  2018280N18273    michael  2018   NaN
14  2018281N12062      luban  2018    NI
15  2018282N09333     nadine  2018   NaN
16  2018282N15087      titli  2018    NI
17  2018288N17257       tara  2018    EP
18  2018292N13269    vicente  2018    EP
19  2018292N14261      willa  2018    EP
20  2018295N08158       yu

In [96]:
#makesure it works with rest of the code 
#drop category column
df0 = df2.copy()
df0.drop(['BASIN'], axis=1, inplace=True)

print('df0 columns :', df0.columns)
# get a dict with the storms with a windspeed and number of timesteps greater to a threshold
storms = proc.sort_storm(df0, min_wind, min_steps)
# pad the trajectories to a fix length
d = proc.pad_traj(storms, max_steps)
# print the shape of the tensor
m, n, t_max, t_min, t_hist = proc.tensor_shape(d)
# create the tensor
t, p_list = proc.create_tensor(d, m)

# #put t in format storm * timestep * features
e = t.transpose((2, 0, 1))
for tt in e:
    try:
        tt[0] = datetime.strptime(tt[0], "%Y-%m-%d %H:%M:%S")
    except:
        pass
t.shape

df0 columns : Index(['SID', 'ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND',
       'STORM_SPEED', 'cos_day', 'sin_day', 'COS_STORM_DIR', 'SIN_STORM_DIR',
       'COS_LAT', 'SIN_LAT', 'COS_LON', 'SIN_LON', 'wind_category',
       'AP01_24_lat', 'AP01_24_lon', 'AP01_24_vmax', 'AP01_24_mslp',
       'SHIP_24_lat', 'SHIP_24_lon', 'SHIP_24_vmax', 'SHIP_24_mslp',
       'CMC_24_lat', 'CMC_24_lon', 'CMC_24_vmax', 'CMC_24_mslp', 'HWRF_24_lat',
       'HWRF_24_lon', 'HWRF_24_vmax', 'HWRF_24_mslp', 'NAM_24_lat',
       'NAM_24_lon', 'NAM_24_vmax', 'NAM_24_mslp', '_basin_EP', '_basin_NI',
       '_basin_SI', '_basin_SP', '_basin_WP'],
      dtype='object')
The dictionary of storms has been created.
The trajectories have now been padded.
There are 4 storms with 42 features, and maximum number of steps is 120 and minimum is 120.
The tensor has now been created.


(120, 42, 3)

In [73]:
# df0.loc[df0['NAME']=='michael']['BASIN']
# df1
# read hurdat data set for both basins: north atlantic and east pacific
hurdat = tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)
# get_forecast(hurdat, 'michael',2018, 24)
# pred=24
storm = hurdat.get_storm(('michael', 2018))
forecast = storm.get_operational_forecasts()
# #choose models 
# model_list = set(['NAM','AP01','HWRF','SHIP', 'AEMI','CLAP5','EMXI','CMC']).intersection(forecast.keys())
# #create empty df 
# df_out = pd.DataFrame(columns=['datetime'])
# for model in model_list:  
#     df_model = pd.DataFrame()
#     for time in forecast[model]:    
#         df = pd.DataFrame(forecast[model][time])
#         temp = df.loc[df['fhr']==pred]
#         #select columns
#         temp = temp[['lat','lon','vmax','mslp']]
#         temp = temp.add_prefix(str(model)+'_'+str(pred)+'_')
#         temp['datetime'] = pd.to_datetime(time, format = '%Y%m%d%H')
#         df_model = pd.concat([df_model, temp], axis=0)
#     df_out = df_out.merge(df_model, on='datetime', how='outer')
# df_out = df_out.sort_values(by='datetime') 

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (11.37 seconds)
